In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [3]:
df = pd.read_csv('train.csv')
# print(df)

In [4]:
y_train = df["number"].iloc[0:40000]
x_train = df.drop(columns='number').iloc[0:40000]
y_test = df["number"].iloc[40000::]
x_test = df.drop(columns='number').iloc[40000::]
# print(x_train.shape)

In [5]:
# mean = np.array(x_train).mean(axis = 0,keepdims=True)
# std = np.array(x_train).std(axis = 0,keepdims=True)

# print(mean.shape,std.shape)
# x_train = (x_train)*(1/10.)
# x_test = (x_test)*(1/10.)

# print(x_train.shape)
# print(x_test.shape)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 196),
            nn.ReLU(),
            nn.Linear(196,196),
            nn.ReLU(),
            nn.Linear(196, 49),
            nn.ReLU(),
            nn.Linear(49, 49),
            nn.ReLU(),
            nn.Linear(49, 10),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.00013, momentum=0.1)

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [10]:

for j in range (0,100):
    for i in range (1,625):
        X = torch.tensor(np.array(x_train.iloc[64*i:64*(i+1)]),dtype=torch.float)*10
        Y = torch.tensor(np.array(y_train.iloc[64*i:64*(i+1)]),dtype=torch.int64)
        Y = torch.nn.functional.one_hot(Y, num_classes=10).type(torch.float)

        pred = model(X)
        loss = loss_fn(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [11]:
size = 2000
print(x_test.shape)
with torch.no_grad():
            X_test = torch.tensor(np.array(x_test),dtype=torch.float)*10
            y_test = torch.tensor(np.array(y_test),dtype=torch.int64)
            Y_test = torch.nn.functional.one_hot(y_test, num_classes=10).type(torch.float)
            pred = model(X_test)
            test_loss = loss_fn(pred, Y_test).item()
            correct = (pred.argmax(1) == y_test).type(torch.float).sum().item()
            
test_loss /= size
correct /= size

print(f"Accuracy: {(100*correct):>0.1f}%")

(2000, 784)
Accuracy: 44.9%


In [12]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.0354,  0.0323,  0.0236,  ..., -0.0210, -0.0242, -0.0175],
        [-0.0021, -0.0044, -0.0135,  ..., -0.0075, -0.0158, -0.0119],
        [-0.0033,  0.0105,  0.0237,  ...,  0.0033,  0.0092,  0.0288],
        ...,
        [ 0.0078,  0.0319,  0.0190,  ...,  0.0143, -0.0054,  0.0312],
        [-0.0022,  0.0119, -0.0333,  ...,  0.0192,  0.0004,  0.0215],
        [ 0.0232, -0.0118,  0.0029,  ..., -0.0258,  0.0085,  0.0060]],
       requires_grad=True), Parameter containing:
tensor([ 0.0096, -0.0166,  0.0107,  0.0041,  0.0305,  0.0212,  0.0346, -0.0345,
        -0.0155, -0.0029, -0.0355,  0.0018,  0.0341,  0.0142, -0.0253, -0.0095,
         0.0235,  0.0253,  0.0302, -0.0117,  0.0261,  0.0135, -0.0180, -0.0210,
        -0.0210, -0.0156, -0.0189, -0.0039, -0.0148, -0.0129, -0.0071, -0.0038,
        -0.0045, -0.0174, -0.0351,  0.0136, -0.0018, -0.0255, -0.0285, -0.0279,
        -0.0034,  0.0183, -0.0345,  0.0189, -0.0310, -0.0208, -0.0147, -0.0294,
        -0.00

In [13]:
# PATH = "state_dict_model.pt"

# Save
# torch.save(model.state_dict(), PATH)

# Load
# model = Net()
# model.load_state_dict(torch.load(PATH))
# model.eval()